# How to manually align some census data with your own data of interest.

In [3]:
import os
import pandas as pd
import requests
from time import time

def get_census_api_key():
    """
    Go here to get a census api key: https://api.census.gov/data/key_signup.html
    Then put it in a file named "census_api_key.txt" in this directory
    The current_path trick is not guaranteed to work in jupyter notebook; hardcode it if you need to
    """
    current_path = os.getcwd()
    with open(os.path.join(current_path, "census_api_key.txt"), "r") as keyfile:
        census_api_key = keyfile.readline().rstrip()
    return census_api_key

census_api_key = get_census_api_key()

#### Load your own dataset of interest into a dataframe
The example data is a public dataset regarding energy and water usage in new york city

In [4]:
mydata_url = "https://data.cityofnewyork.us/resource/usc3-8zwd.csv"
my_df = pd.read_csv(mydata_url)
print(f"Read {my_df.shape[0]} rows from dataset of interest")

Read 1000 rows from dataset of interest


### Identify the column with geographical data, like zipcode, city, county, lat/long
Create a duplicate of that column that's all strings, for later simplicity

In [15]:
# Look for something like zipcode or postcode in the column names
print(f"Potential columns of interest: {my_df.filter(regex='code|county|zip').columns}")

# When you find one, put it here
mydata_geo_column = "postcode"
mydata_geo_column_str = f"{mydata_geo_column}_str"
my_df[mydata_geo_column_str] = my_df[mydata_geo_column].apply(lambda x: str(x))
print(f"geo_column looks like: \n{my_df[mydata_geo_column_str][:10]}")

Potential columns of interest: Index(['postcode', 'postcode_str'], dtype='object')
geo_column looks like: 
0    10036
1    10017
2    10036
3    10017
4    10036
5    10032
6    10021
7    10034
8    10022
9    10005
Name: postcode_str, dtype: object


### Identify a census database that you want to use
Plan is to make an interactive module to help with this as part of this library

For now you'll have to search here: https://api.census.gov/data.html

Take note of:

1. the database's url
1. the name of the variable of interest
1. the name of the column with geographic info that you'll use for alignment

In [8]:
census_database_url = "http://api.census.gov/data/1994/zbp"
census_variable = "PAYQTR1"
census_geo_column = "zipcode"

### Generate the predicate part of the query
The predicate limits the query to only certain rows of the data, so you're not retrieving the whole table.

Typically, you'll want to use your geographic column to find associated rows in the census data. That is, for each (zipcode, county, region) in your data of interest, you want to find and grab that row from the census data. 

You don't want to use a `for` loop for this task, i.e. moving through each row of your data to make a request based on that row. Sometimes the census API is very slow - a minute or more per request. Think about what is the info from you need from your data to create the constraint in the census data, and collect it up. Don't worry about what order it's in.

Here, we get the `set` of all `postcode`s in our data of interest. There are fewer than 200 unique ones in the 1000 rows we downloaded. We turn these into a comma-separated string and prefix it with `zipcode:`, because that's the column in the census data that contains the same info. 

The set of comma-separated postcodes/zipcodes get slotted into the part of the request that says "&for=" -- the predicate. 

It can take a few tries to get this right when you're starting fresh. Use the api guide (linked below) for help.


In [9]:
census_predicate = f"zipcode:{','.join([i for i in set(my_df['postcode_str'])])}"

### Make the http request
Build up the request from the variables above, using https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf

It takes about 40 seconds for the request to complete.

In [10]:
request = f"{census_database_url}?get={census_variable}&for={census_predicate}&key={census_api_key}"
print(f"Sending request (abbreviated): {request[:150]}")
t1 = time()
response = requests.get(request)
t2 = time()
print(f"Response received in {(t2-t1)} seconds")

Sending request (abbreviated): http://api.census.gov/data/1994/zbp?get=PAYQTR1&for=zipcode:10004,11365,11428,11379,11357,11367,10304,10301,10034,11429,11366,11417,10006,10128,10456,
Response received in 42.77474236488342 seconds


### Check the response status code
If you try to parse it right away, it can be difficult to understand error codes

So first check for an errorful response, and try to understand that. https://developer.mozilla.org/en-US/docs/Web/HTTP/Status#successful_responses

1. error 400: bad request, the query is malformed
1. error 204; no content in the response (good request but no corresponding entries?)
1. status 200: good to go!


In [11]:
if response.status_code != 200:
    print(f"Request returned an error response, status code: {response.status_code}")
else:
    print(f"Census data retrieved, proceed with parsing")

Census data retrieved, proceed with parsing


### Create a dataframe of the data you retrieved
It's a one-liner!

In [12]:
census_df = pd.DataFrame(response.json()[1:], columns=response.json()[0])
print(f"Retrieved {census_df.shape[0]} rows with columns {census_df.columns}")

Retrieved 182 rows with columns Index(['PAYQTR1', 'zipcode'], dtype='object')


### Align the two datasets using the geographic data in each
Again, a one-liner does the trick.

Make a new column in your dataframe that contains the correct value of interest for each value in your data's geographic column. We use the `pandas.DataFrame.join` method - no `for` loops!

In [13]:
my_df_enhanced = my_df.join(census_df.set_index(census_geo_column), on=mydata_geo_column_str)
print("Databases joined, items of interest:")    
print(my_df_enhanced[[mydata_geo_column, census_variable]])

Databases joined, items of interest:
     postcode  PAYQTR1
0       10036  1116756
1       10017  1723647
2       10036  1116756
3       10017  1723647
4       10036  1116756
..        ...      ...
995     11235    77067
996     11225    13276
997     11230    52179
998     11230    52179
999     11203   130846

[1000 rows x 2 columns]


You're now free to use your enhanced dataset to model the demographic variable against the data you collected elsewhere. You can also repeat the above to get additional variables from the census data and align it the same way - just change the `census_variable` entry and rerun.